In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2



In [8]:
# 计算两个经纬度坐标之间的距离，这里采用欧几里得距离
def distance(coord1, coord2):
    R = 6371e3  # 地球半径，单位：米
    lat1, lon1, lat2, lon2 = map(radians, [coord1[1], coord1[0], coord2[1], coord2[0]])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

# 找到距离给定坐标最近的n个信标
def find_nearest_beacons(df, coord, n):
    df['distance'] = df.apply(lambda row: distance(coord, [row['x'], row['y']]), axis=1)
    nearest_beacons = df.nsmallest(n, 'distance')
    return nearest_beacons

def rssi_distance_relation(x):
    n = -2.38368781
    d0 = -49.71997549
    return 10*n * np.log10(x) + d0

# 测试数据
df =  pd.read_csv(r'C:\\Users\Administrator\Desktop\WSN+IMU定位\\数据\\测试集\Beacon.csv')

# 给定坐标
coord_A = [114.4719753,30.46045143]

# 要找的信标数量
num_beacons = 4

# 调用函数
nearest_beacons = find_nearest_beacons(df, coord_A, num_beacons)
nearest_beacons['RSSI'] = nearest_beacons['distance'].apply(rssi_distance_relation)
noise = np.random.normal(0, 1, len(nearest_beacons['RSSI']))
nearest_beacons['RSSI_noise'] = nearest_beacons['RSSI'] + noise
# 输出结果

print(nearest_beacons)

       id           x          y   distance       RSSI  RSSI_noise
90   B091  114.472030  30.460411   6.902835 -69.719771  -67.772172
138  B139  114.471811  30.460411  16.372341 -78.660586  -78.624517
137  B138  114.472030  30.460609  18.291341 -79.807969  -80.164163
44   B045  114.471811  30.460609  23.557952 -82.427451  -83.426703
